*Read the data 

In [147]:
import pandas as pd
import mysql.connector
from datetime import datetime

#step 1: Load the CSV file
file_path=r"C:\Users\Balasubramanian\Downloads\policelog\traffic_stops - traffic_stops_with_vehicle_number.csv"
df = pd.read_csv(file_path, low_memory=False)

In [148]:
print(df.dtypes)



stop_date             object
stop_time             object
country_name          object
driver_gender         object
driver_age_raw         int64
driver_age             int64
driver_race           object
violation_raw         object
violation             object
search_conducted        bool
search_type           object
stop_outcome          object
is_arrested             bool
stop_duration         object
drugs_related_stop      bool
vehicle_number        object
dtype: object


In [149]:
print(df.isnull().sum())


stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64


In [150]:
print(f"Total rows in dataset: {len(df)}")

Total rows in dataset: 65538


In [151]:
#step 2: Data Cleaning

# Drop columns with missing values
df.dropna(axis=1, how='any', inplace=True)
# Fill NaN values with suitable defaults
df.fillna({
    'driver_age':df['driver_age'].median(),
    'search_type': 'None',
    'stop_duration': 'Unknown',
    'violation': 'Unknown',
    'stop_outcome': 'Unknown',
}, inplace=True)

# Convert to proper datetime object (date only)
df['stop_date'] = pd.to_datetime(df['stop_date'], format='%Y-%m-%d', errors='coerce')

# Convert time part to datetime.time
df['stop_time'] = pd.to_datetime(df['stop_time'], format='%H:%M:%S', errors='coerce').dt.time


 

In [152]:
print(type(df['stop_time'].iloc[0]))  # Should show: <class 'datetime.time'>

<class 'datetime.time'>


In [153]:
df['stop_timestamp'] = pd.to_datetime(
    df['stop_date'].astype(str) + ' ' + df['stop_time'].astype(str),
    errors='coerce'
)
# Confirm the result
print(df[['stop_date', 'stop_time']].head())
print(df.dtypes)

   stop_date stop_time
0 2020-01-01  00:00:00
1 2020-01-01  00:01:00
2 2020-01-01  00:02:00
3 2020-01-01  00:03:00
4 2020-01-01  00:04:00
stop_date             datetime64[ns]
stop_time                     object
country_name                  object
driver_gender                 object
driver_age_raw                 int64
driver_age                     int64
driver_race                   object
violation_raw                 object
violation                     object
search_conducted                bool
stop_outcome                  object
is_arrested                     bool
stop_duration                 object
drugs_related_stop              bool
vehicle_number                object
stop_timestamp        datetime64[ns]
dtype: object


In [154]:
print(df.isnull().sum())

stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age_raw        0
driver_age            0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
stop_timestamp        0
dtype: int64


In [155]:
print(f"Total rows in dataset: {len(df)}")

Total rows in dataset: 65538


In [156]:
import psycopg2
import pandas as pd

In [157]:
#connect to postgresql database
import psycopg2

# Establish connection
connection = psycopg2.connect(
    host="localhost",                # Server address
    database="secure_police_db",     # Database name
    user="postgres",                 # Your username
    password="10987654321"           # Your password
)

# Create a cursor object for executing queries
cursor = connection.cursor()

# Test the connection with a simple query
cursor.execute("SELECT version();")
print("Connected to:", cursor.fetchone())

Connected to: ('PostgreSQL 17.5 on x86_64-windows, compiled by msvc-19.44.35209, 64-bit',)


In [158]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS police_logs (
    id SERIAL PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(50),
    stop_outcome VARCHAR(100),
    drugs_related_stop BOOLEAN,
    stop_duration VARCHAR(50),
    vehicle_number VARCHAR(50),
    timestamp TIMESTAMP
)
""")
connection.commit()

In [159]:
cursor.execute("SELECT COUNT(*) FROM police_logs;")
print(f"🧮 Rows currently in police_logs: {cursor.fetchone()[0]}")


🧮 Rows currently in police_logs: 0


In [160]:
total_missing = df.isnull().sum().sum()
print(f"Total missing entries in the dataset: {total_missing}")


Total missing entries in the dataset: 0


In [161]:
#Replace NaN values with None
df = df.where(pd.notnull(df), None)

In [162]:

# Insert data into the PostgreSQL table
insert_query = """
INSERT INTO police_logs (
    stop_date, stop_time, country_name, driver_gender, driver_age, 
    driver_race,violation, search_conducted, search_type, 
    stop_outcome, drugs_related_stop, stop_duration, vehicle_number, timestamp
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [163]:
#Iterate over the DataFrame and insert each row
for _, row in df.iterrows():
    cursor.execute(insert_query, (
        row['stop_date'],
        row['stop_time'],
        row.get('country_name'),
        row.get('driver_gender'),
        int(row['driver_age']) if pd.notnull(row['driver_age']) else None,
        row.get('driver_race'),
        row.get('violation'),
        row['search_conducted'] if pd.notnull(row['search_conducted']) else None,
        row.get('search_type'),
        row.get('stop_outcome'),
        row['drugs_related_stop'] if pd.notnull(row['drugs_related_stop']) else None,
        row.get('stop_duration'),
        row['vehicle_number'] if pd.notnull(row['vehicle_number']) else f"VEH{row.name}",
        row['stop_timestamp']
    ))

connection.commit()

In [164]:
print(f"Total rows: {df.shape[0]}")


Total rows: 65538


In [165]:
print("Raw CSV rows:", pd.read_csv(file_path).shape[0])
print("After cleaning:", df.shape[0])


Raw CSV rows: 65538
After cleaning: 65538


C:\Users\Balasubramanian\AppData\Local\Temp\ipykernel_17808\2291146326.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  print("Raw CSV rows:", pd.read_csv(file_path).shape[0])
